In [23]:
# time_constraint = 15000
# path_raw = "ID: 0->ID: 3->ID: 8->ID: 10->ID: 16->ID: 9->ID: 6->ID: 2->ID: 1->ID: 4->ID: 17->ID: 26->ID: 35->ID: 37->ID: 34->ID: 28->ID: 12->ID: 22->ID: 25->ID: 19->ID: 18->ID: 20->ID: 23->ID: 36->ID: 21->ID: 13->ID: 29->ID: 15->ID: 14->ID: 41->ID: 42->ID: 38->ID: 5->ID: 11->ID: 7->ID: 0->"
# legend_placement = [52.532883, 13.339717]

time_constraint = 30000
path_raw = "ID: 0->ID: 3->ID: 8->ID: 10->ID: 16->ID: 9->ID: 6->ID: 2->ID: 1->ID: 4->ID: 17->ID: 26->ID: 35->ID: 37->ID: 34->ID: 28->ID: 12->ID: 22->ID: 25->ID: 19->ID: 18->ID: 20->ID: 23->ID: 36->ID: 21->ID: 13->ID: 29->ID: 15->ID: 14->ID: 41->ID: 42->ID: 38->ID: 5->ID: 11->ID: 7->ID: 30->ID: 24->ID: 32->ID: 33->ID: 27->ID: 31->ID: 40->ID: 46->ID: 45->ID: 48->ID: 56->ID: 52->ID: 62->ID: 58->ID: 68->ID: 65->ID: 73->ID: 50->ID: 51->ID: 94->ID: 0->"
legend_placement = [52.566917, 13.308067]

In [24]:
import json

with open("matrix500.json", 'r') as infile:
    data = json.load(infile)

geocaches = [(lat, lon) for lon, lat in data['metadata']['query']['locations']]

In [25]:
path = [int(val) for val in list(filter(None, path_raw.replace('ID: ', '').split('->')))]

print(path)

[0, 3, 8, 10, 16, 9, 6, 2, 1, 4, 17, 26, 35, 37, 34, 28, 12, 22, 25, 19, 18, 20, 23, 36, 21, 13, 29, 15, 14, 41, 42, 38, 5, 11, 7, 30, 24, 32, 33, 27, 31, 40, 46, 45, 48, 56, 52, 62, 58, 68, 65, 73, 50, 51, 94, 0]


In [26]:
import folium
from algo_results import create_marker, create_circle_marker, fit_locations, style_function, create_legend

In [27]:
import math
max_geocaches = int(math.ceil((max(path) + 1) / 10.0)) * 10

In [28]:
map = folium.Map(tiles=None, location=geocaches[path[0]], zoom_start=14, zoom_control=False)
folium.TileLayer(tiles='Stamen Toner', opacity=0.15).add_to(map)
pass

In [29]:
from context import openroute

optimal_locations = [(geocaches[id][1], geocaches[id][0]) for id in path]

request = {
    'coordinates': optimal_locations,
    'geometry': True,
    'instructions': False
}
optimal_route = openroute.request_directions(request, format_out='/geojson')
folium.features.GeoJson(data=optimal_route,
                        name='Optimal route',
                        style_function=style_function('#bb616b'),
                        overlay=True).add_to(map)

fit_locations(map, geocaches[:max_geocaches])

In [30]:
for index, coords in enumerate(geocaches[:max_geocaches+100]):
    color = '#787c99'
    if index == path[0]: # start
        color = '#41a6b5'
    elif index == path[-2]: # last node before start
        color = '#9d7cd8'
    elif index in path: # other visited
        color = '#3c7da2'
    create_marker(path, coords, index, color, radius=8).add_to(map)

In [31]:
import datetime

optimal_duration = optimal_route['features'][0]['properties']['summary']['duration'] / 60

legend = create_legend(legend_placement, html=f"""
    <table width=100%>
        <tr>
            <td>Visited nodes:</td>
            <td align=right>{len(path)-1}</td>
        </tr>
        <tr>
            <td>Total duration:</td>
            <td align=right>{str(datetime.timedelta(seconds=round(optimal_duration*60)))}</td>
        </tr>
        <tr>
            <td>Time constraint:</td>
            <td align=right>{str(datetime.timedelta(seconds=time_constraint))}</td>
        </tr>
    </table>
    <div class="marker-description">
        <span class="start">Start node</span>
        <span class="last">Penultimate node</span>
    </div>
""").add_to(map)

map